In [1]:
# Data Management 
using DataFrames, DataFramesMeta
using DrWatson

# Statistics
using Random
using Distributions
using StatsBase

# Graphs 
using LightGraphs, SimpleWeightedGraphs, SimpleHypergraphs, GraphIO
using GraphPlot

# Modelling
using Agents
include("./ElectoralPhysics.jl")

# Data Visualization
using Plots
using AgentsPlots
using PlotThemes

# Python
# ENV["PYTHON"] = "path/to/python"
# Pkg.build("PyCall")
using PyCall
#using PyPlot
nx = pyimport("networkx")
hnx= pyimport("hypernetx")
np = pyimport("numpy")
#hvnx = pyimport("hvplot.networkx")
nw = pyimport("netwulf");

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260
┌ Info: Precompiling DataFramesMeta [1313f7d8-7da2-5740-9ea0-a2ca25f37964]
└ @ Base loading.jl:1260
┌ Info: Precompiling DrWatson [634d3b9d-ee7a-5ddf-bec9-22491ea816e1]
└ @ Base loading.jl:1260
┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1260
┌ Info: Precompiling LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1260
┌ Info: Precompiling SimpleWeightedGraphs [47aef6b3-ad0c-573a-a1e2-d07658019622]
└ @ Base loading.jl:1260
┌ Info: Precompiling SimpleHypergraphs [aa4a32ff-dd5d-5357-90e3-e7a9512f0501]
└ @ Base loading.jl:1260
┌ Info: Precompiling GraphIO [aa1b3936-2fda-51b9-ab35-c553d3a640a2]
└ @ Base loading.jl:1260
┌ Info: Precompiling GraphPlot [a2cc645c-3eea-5389-862e-a155d0052231]
└ @ Base loading.jl:1260
┌ Info: Precompiling Agents [46ada45e-f475-11e8-01d0-f70cc89e6671]
└ @ Base loading.jl:1260
┌ Info: Precompiling Plot

### Agent Type

In [2]:
# Voter Definition
mutable struct Voter <: AbstractAgent
    id::Int64             # ∈ ℕ
    pos::Int64            # ∈ ℕ
    candidacy::Bool       # ∈ 𝔹
    score::Float64        # ∈ 𝔹
    final_score::Float64  # \in \bbR
    community::Int64      # ∈ {1,2,3, ..., n}
end

### Functions

In [3]:
function vote!(agent,model)
    
    if agent.candidacy == true
        return
    end
    
    candidates = [agent for agent in allagents(model) if agent.candidacy == true]
    nvotes = nvotes!(agent, model)
    targets = preferential_attachment_choose_candidates!(agent, model, candidates, nvotes)
    for target in targets
        add_edge!(model.space.graph, agent.pos, target.pos, 1/nvotes)
    end
end

function nvotes!(agent, model)
    p = 0.1
    if agent.candidacy == true
        d = Binomial(model.properties[:M]-2,p)
        nvotes = rand(d)
    else 
        d = Binomial(model.properties[:M]-1,p)
        nvotes = rand(d)
    end
    
    return nvotes + 1
end

function choose_candidates!(agent, model, candidates, nvotes)
    targets = []
    for vote in 1:nvotes
        target = rand(candidates)
        while target.id == agent.id || target in targets
            target = rand(candidates)
        end
        push!(targets, target)
    end
    return targets
end

function preferential_attachment_choose_candidates!(agent, model, candidates, nvotes)
    agent_community = agent.community
    weights = Int64[]
    for candidate in candidates
        if candidate.community == agent_community
            push!(weights, 10)
        else
            push!(weights, 1)
        end
    end
    
    targets = []
    for vote in 1:nvotes
        target = StatsBase.sample(candidates, ProbabilityWeights(weights))
        while target.id == agent.id || target in targets
            target = StatsBase.sample(candidates, ProbabilityWeights(weights))
        end    
        push!(targets, target)
    end
    
    return targets
end


# Voter Dynamics 
function agent_step!(agent, model)
    vote!(agent, model)
end;

### Updates

In [4]:
function update_score_voters!(model)
    candidates = [agent for agent in allagents(model) if agent.candidacy == true]
    hypergraph = model.properties[:H]
    for candidate in candidates
        votes = [vote for vote in hypergraph[candidate.id,:] if vote != nothing]
        candidate.score = candidate.score + sum(votes)
    end
end

function update_score_flow!(model)
    for agent in allagents(model)
        if agent.candidacy == true
            for inneighbor in node_neighbors(agent, model; neighbor_type=:in)
                agent_inneighbor = get_node_agents(inneighbor, model)[1]
                if agent_inneighbor.candidacy == false
                    agent.score = agent.score + agent_inneighbor.score * LightGraphs.weights(model.space.graph)[agent_inneighbor.pos, agent.pos]
                end
            end
        end
    end
end

function update_score_pagerank!(model, α, n, ϵ)
    for agent in allagents(model)
        agent.score = pagerank(model.space.graph, α, n, ϵ)[agent.id]
    end
end;

### Model Inizialization

In [5]:
# Voter Model Initialization
function init_model_voters(N::Int64, M::Int64, K::Int64, community)
       
    properties = @dict(N, M, K)
    
    # Create Directed Weighted Graph
    space = GraphSpace(SimpleWeightedDiGraph(N+M)) 
    model_voters = ABM(Voter, space; properties = properties)
    
    
    for id in 1:M
        pos = id
        score = 1.0
        final_score = 0
        candidacy = true
        add_agent!(pos, model_voters, candidacy, score, final_score, community[id])
    end

    for id in (M+1):N+M
        pos = id
        score = 1.0
        final_score = 0
        candidacy = false
        add_agent!(pos, model_voters, candidacy, score, final_score, community[id])
    end
    
    return model_voters
end;

### Parameters

In [6]:
# Create population of voters 
N = 80

# Create subpopulation of candidates
M = 10;

# Create subpopulation of elected
K = 3;

# Initialize number of steps
nsteps = 1;



community = []
for i in 1:N
    if i <= M
        if i < M/3
            push!(community,1)
        elseif i >= M/3 && i < 2*(M/3)
            push!(community,2)
        else
            push!(community,3)
        end
    else
         if i < N/3
            push!(community,1)
        elseif i >= N/3 && i < 2*(N/3)
            push!(community,2)
        else
            push!(community,3)
        end
    end
end
append!(community, community[1:M])


α = 0.85
n = 500 
ϵ = 1e-6;

### Voter Simulation

In [7]:
# Seed Selection
Random.seed!(1234);

# Model Instantiation
model_voters = init_model_voters(N, M, K, community)

# Data Collection
data_voters = run!(model_voters, agent_step!, nsteps);

#update_score_flow!(model_voters)

sort!(DataFrame(allagents(model_voters)), :score, rev = true)

,id,pos,candidacy,score,final_score,community
,Int64,Int64,Bool,Float64,Float64,Int64
1,68,68,0,1.0,0.0,3
2,2,2,1,1.0,0.0,1
3,89,89,0,1.0,0.0,3
4,11,11,0,1.0,0.0,1
5,39,39,0,1.0,0.0,2
6,46,46,0,1.0,0.0,2
7,85,85,0,1.0,0.0,2
8,25,25,0,1.0,0.0,1
9,55,55,0,1.0,0.0,3


In [ ]:
### Candidates 

### Visualization

In [8]:
V = model_voters.space.graph
V = ElectoralPhysics.LightGraphs_to_NetworkX(V);
V = nx.convert_node_labels_to_integers(V, first_label=1);
nx.write_graphml(V, "bipartite_graph.graphml")

In [9]:
is_bipartite(model_voters.space.graph)

true

In [10]:
model_voters.space.graph

{90, 161} directed simple Int64 graph with Float64 weights

In [19]:
outdegree(model_voters.space.graph)[11]

3